# Mexican Federal Budget
This notebook aims at matching column names across years.

## 1. PEF files (approved budgets)

### Load files

In [1]:
from pandas import read_csv 

def load_csv_files(years, file_format):
    df = {}
    for year in years:
        print('Loading', year)
        df[year] = read_csv(file_format % year, encoding='iso-8859-1')
    return df

In [2]:
pef_years = range(2008, 2017)
pef_file_format = 'PEF%s_AC01.csv'
pef = load_csv_files(pef_years, pef_file_format)

Loading 2008


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2809: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Loading 2009
Loading 2010
Loading 2011
Loading 2012
Loading 2013
Loading 2014
Loading 2015


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2809: DtypeWarning: Columns (3,25) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Loading 2016


### Clean up columns

#### Delete unnamed columns

In [3]:
def delete_empty_columns(batch):
    for year in batch.keys():
        for column in batch[year].columns:
            if 'Unnamed:' in column:
                try:
                    print('deleting', year, column)
                    del batch[year][column]
                except KeyError:
                    pass

In [4]:
delete_empty_columns(pef)

deleting 2008 Unnamed: 24
deleting 2008 Unnamed: 25
deleting 2008 Unnamed: 26
deleting 2008 Unnamed: 27
deleting 2008 Unnamed: 28
deleting 2008 Unnamed: 29
deleting 2008 Unnamed: 30
deleting 2009 Unnamed: 24
deleting 2009 Unnamed: 25
deleting 2009 Unnamed: 26
deleting 2009 Unnamed: 27
deleting 2009 Unnamed: 28
deleting 2009 Unnamed: 29
deleting 2009 Unnamed: 30
deleting 2010 Unnamed: 24
deleting 2010 Unnamed: 25
deleting 2010 Unnamed: 26
deleting 2010 Unnamed: 27
deleting 2010 Unnamed: 28
deleting 2010 Unnamed: 29
deleting 2010 Unnamed: 30
deleting 2011 Unnamed: 24
deleting 2011 Unnamed: 25
deleting 2011 Unnamed: 26
deleting 2011 Unnamed: 27
deleting 2011 Unnamed: 28
deleting 2011 Unnamed: 29
deleting 2011 Unnamed: 30
deleting 2012 Unnamed: 24
deleting 2012 Unnamed: 25
deleting 2012 Unnamed: 26
deleting 2012 Unnamed: 27
deleting 2012 Unnamed: 28
deleting 2012 Unnamed: 29
deleting 2012 Unnamed: 30


#### Strip blanks in column names
Some column names have spaces on the edges. This needs to be taken care of before we proceed.

In [5]:
def strip_blanks(batch):
    for year in batch.keys():
        for column in batch[year].columns:
            batch[year].rename(columns={column: column.strip()}, inplace=True)
    return batch

In [6]:
pef = strip_blanks(pef)

### Compute the union of all column names

In [7]:
def get_union_of_columns(batch):
    union = set()
    for year in batch.keys():
        union = union | set(batch[year].columns)
    return union

In [8]:
union_of_pef_columns = get_union_of_columns(pef)
union_of_pef_columns

{'ACTIVIDAD_INST_DESCRIPCION',
 'AI',
 'Actividad Institucional',
 'CLAVE_CARTERA',
 'CONCEPTO',
 'CONCEPTO_DESCRIPCION',
 'Ciclo',
 'Clave de Cartera',
 'Descripción de Actividad Institucional',
 'Descripción de Entidad Federativa',
 'Descripción de Finalidad',
 'Descripción de Fuente de Financiamiento',
 'Descripción de Función',
 'Descripción de Grupo Funcional',
 'Descripción de Objeto del Gasto',
 'Descripción de Programa Presupuestario',
 'Descripción de Ramo',
 'Descripción de Subfunción',
 'Descripción de Tipo de Gasto',
 'Descripción de Unidad Responsable',
 'Descripción de la Actividad Institucional',
 'Descripción del Identificador de Programa Presupuestario',
 'Descripción del Identificador del Programa Presupuestario',
 'Descripciónde Fuente de Financiamiento',
 'Descripciónde Tipo de Gasto',
 'EF',
 'ENTIDAD_FED_DESCRIPCION',
 'Entidad Federativa',
 'FF',
 'FN',
 'FUENTE_FINAN_DESCRIPCION',
 'FUNCIONL_DESCRIPCION',
 'Finalidad',
 'Fuente de Financiamiento',
 'Función',
 '

### Column aliases

We have prepared, by hand, a YAML file containing all possible aliases for each column. Let's load it.

In [9]:
from yaml import load
from json import dumps

def load_aliases(file):
    with open(file) as yaml:
        return load(yaml.read())

In [10]:
pef_column_aliases = load_aliases('column_aliases.pef.yaml')
pef_column_aliases

{'ACTIVIDAD_INST_DESCRIPCION': ['Descripción de Actividad Institucional',
  'Descripción de la Actividad Institucional'],
 'AI': ['Actividad Institucional'],
 'CLAVE_CARTERA': ['Clave de Cartera'],
 'CONCEPTO': ['Objeto del Gasto'],
 'CONCEPTO_DESCRIPCION': ['Descripción de Objeto del Gasto'],
 'Ciclo': None,
 'EF': ['Entidad Federativa'],
 'ENTIDAD_FED_DESCRIPCION': ['Descripción de Entidad Federativa'],
 'FF': ['Fuente de Financiamiento'],
 'FN': ['Función'],
 'FUENTE_FINAN_DESCRIPCION': ['Descripción de Fuente de Financiamiento',
  'Descripción Fuente de Financiamiento',
  'Descripciónde Fuente de Financiamiento'],
 'FUNCIONL_DESCRIPCION': ['Descripción de Función'],
 'GF': ['Finalidad', 'Grupo Funcional'],
 'GRUPO_FUN_DESCRIPCION': ['Descripción de Finalidad',
  'Descripción de Grupo Funcional'],
 'MOD': ['Modalidad del Programa presupuestario',
  'Identificador de Programa Presupuestario',
  'Identificador del Programa Presupuestario'],
 'MODALIDAD_DESCRIPCION': ['Descripción de l

### Unify column nomenclature

Now use the list of aliases in the YAML file to unify the column nomenclature across all years.

In [11]:
from copy import deepcopy

def alias_columns(input_batch, list_of_aliases):
    batch = deepcopy(input_batch)

    for year in sorted(batch.keys()):
        for column in sorted(batch[year].columns):
            if not column in list_of_aliases:
                for reference, aliases in list_of_aliases.items():
                    if aliases:
                        if column in aliases:
                            batch[year].rename(columns={column: reference}, inplace=True)
                            print(year, 'REPLACING:', column, ' -> ', reference)
                            break  
                else:
                    print(year, 'NO ALIAS: ', column)
        print()
        
    return batch

In [12]:
pef_aliased = alias_columns(pef, pef_column_aliases)

2008 REPLACING: Actividad Institucional  ->  AI
2008 REPLACING: Descripción de Actividad Institucional  ->  ACTIVIDAD_INST_DESCRIPCION
2008 REPLACING: Descripción de Fuente de Financiamiento  ->  FUENTE_FINAN_DESCRIPCION
2008 REPLACING: Descripción de Función  ->  FUNCIONL_DESCRIPCION
2008 REPLACING: Descripción de Grupo Funcional  ->  GRUPO_FUN_DESCRIPCION
2008 REPLACING: Descripción de Objeto del Gasto  ->  CONCEPTO_DESCRIPCION
2008 REPLACING: Descripción de Programa Presupuestario  ->  PROGR_PRES_DESCRIPCION
2008 REPLACING: Descripción de Ramo  ->  RAMO_DESCRIPCION
2008 REPLACING: Descripción de Subfunción  ->  SUBFUNCIONL_DESCRIPCION
2008 REPLACING: Descripción de Tipo de Gasto  ->  TIPO_GASTO_DESCRIPCION
2008 REPLACING: Descripción de Unidad Responsable  ->  UNIDAD_DESCRIPCION
2008 REPLACING: Descripción del Identificador de Programa Presupuestario  ->  MODALIDAD_DESCRIPCION
2008 REPLACING: Fuente de Financiamiento  ->  FF
2008 REPLACING: Función  ->  FN
2008 REPLACING: Grupo Func

In [13]:
get_union_of_columns(pef_aliased)

{'ACTIVIDAD_INST_DESCRIPCION',
 'AI',
 'CLAVE_CARTERA',
 'CONCEPTO',
 'CONCEPTO_DESCRIPCION',
 'Ciclo',
 'EF',
 'ENTIDAD_FED_DESCRIPCION',
 'FF',
 'FN',
 'FUENTE_FINAN_DESCRIPCION',
 'FUNCIONL_DESCRIPCION',
 'GF',
 'GRUPO_FUN_DESCRIPCION',
 'MOD',
 'MODALIDAD_DESCRIPCION',
 'PEF_2016',
 'PP',
 'PROGR_PRES_DESCRIPCION',
 'RA',
 'RAMO_DESCRIPCION',
 'REASIGNACION_DESCRIPCION',
 'Ramo',
 'SF',
 'SUBFUNCIONL_DESCRIPCION',
 'TG',
 'TIPO_GASTO_DESCRIPCION',
 'UNIDAD',
 'UNIDAD_DESCRIPCION'}

### Overview of file columns

Produce a table summarizing which year contains which columns.

In [14]:
from pandas import DataFrame

def build_overview(batch):
    table = []
    
    for column in get_union_of_columns(batch):
        row = {'Column': column}
        for year in batch.keys():
            row.update({year: column in batch[year].columns})
        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    
    return DataFrame(table).reindex_axis(ordered_columns, axis=1)

In [15]:
pef_columns_overview = build_overview(pef_aliased)
pef_columns_overview

,Column,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,RA,False,False,False,False,False,False,False,False,True
1,UNIDAD_DESCRIPCION,True,True,True,True,True,True,True,True,True
2,PROGR_PRES_DESCRIPCION,True,True,True,True,True,True,True,True,True
3,CONCEPTO,True,True,True,True,True,True,True,True,True
4,REASIGNACION_DESCRIPCION,False,False,False,False,False,False,False,False,True
5,FF,True,True,True,True,True,True,True,True,True
6,ACTIVIDAD_INST_DESCRIPCION,True,True,True,True,True,True,True,True,True
7,RAMO_DESCRIPCION,True,True,True,True,True,True,True,True,True
8,PEF_2016,True,True,True,True,True,True,True,True,True
9,TIPO_GASTO_DESCRIPCION,True,True,True,True,True,True,True,True,True


Save the table to a CSV file.

In [16]:
pef_columns_overview.to_csv('pef.columns.aliased.csv', index=False, encoding='utf-8')

In [17]:
ls -lh| grep pef.columns.aliased.csv

-rw-rw-r-- 1 loic loic 1.8K Aug  8 20:39 pef.columns.aliased.csv


In [18]:
cat pef.columns.aliased.csv | head -n 3

Column,2008,2009,2010,2011,2012,2013,2014,2015,2016
RA,False,False,False,False,False,False,False,False,True
UNIDAD_DESCRIPCION,True,True,True,True,True,True,True,True,True


## 2.  Cuenta Publica files

In [21]:
cp_years = range(2010, 2016)
cp_file_format = 'Cuenta_Publica_%s.csv'

cp = load_csv_files(cp_years, cp_file_format)

Loading 2010
Loading 2011
Loading 2012


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2809: DtypeWarning: Columns (3,25) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Loading 2013
Loading 2014
Loading 2015


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2809: DtypeWarning: Columns (3,25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [22]:
delete_empty_columns(cp)

deleting 2011 Unnamed: 25
deleting 2011 Unnamed: 26
deleting 2011 Unnamed: 27
deleting 2011 Unnamed: 28
deleting 2011 Unnamed: 29
deleting 2011 Unnamed: 30
deleting 2011 Unnamed: 31
deleting 2011 Unnamed: 32
deleting 2011 Unnamed: 33
deleting 2011 Unnamed: 34
deleting 2011 Unnamed: 35
deleting 2011 Unnamed: 36
deleting 2011 Unnamed: 37
deleting 2011 Unnamed: 38
deleting 2011 Unnamed: 39
deleting 2011 Unnamed: 40
deleting 2011 Unnamed: 41


In [23]:
cp = strip_blanks(cp)

In [24]:
union_of_cp_columns = get_union_of_columns(cp)
union_of_cp_columns

{'ADEFAS',
 'Actividad Institucional',
 'Adefas',
 'Aprobado',
 'Ciclo',
 'Clave de cartera',
 'Descripción de Finalidad',
 'Descripción de Fuente de Financiamiento',
 'Descripción de Función',
 'Descripción de Grupo Funcional',
 'Descripción de Objeto del Gasto',
 'Descripción de Programa Presupuestario',
 'Descripción de Ramo',
 'Descripción de Subfunción',
 'Descripción de Tipo de Gasto',
 'Descripción de Unidad Responsable',
 'Descripción de la Actividad Institucional',
 'Descripción de la entidad federativa',
 'Descripción de la modalidad del programa presupuestario',
 'Devengado',
 'Ejercicio',
 'Ejercido',
 'Entidad Federativa',
 'Finalidad',
 'Fuente de Financiamiento',
 'Función',
 'Grupo Funcional',
 'Modalidad del Programa presupuestario',
 'Modificado',
 'Objeto del Gasto',
 'Pagado',
 'Programa Presupuestario',
 'Ramo',
 'Subfunción',
 'Tipo de Gasto',
 'Unidad Responsable'}

In [25]:
cp_column_aliases = load_aliases('column_aliases.cuenta_publica.yaml')

In [26]:
cp_aliased = alias_columns(cp, cp_column_aliases)

2010 REPLACING: Descripción de Finalidad  ->  Descripción de Grupo Funcional
2010 REPLACING: Finalidad  ->  Grupo Funcional

2011 REPLACING: Descripción de Finalidad  ->  Descripción de Grupo Funcional
2011 REPLACING: Finalidad  ->  Grupo Funcional

2012 REPLACING: Descripción de Finalidad  ->  Descripción de Grupo Funcional
2012 REPLACING: Finalidad  ->  Grupo Funcional

2013 REPLACING: Descripción de Finalidad  ->  Descripción de Grupo Funcional
2013 REPLACING: Finalidad  ->  Grupo Funcional

2014 REPLACING: ADEFAS  ->  Adefas




In [27]:
get_union_of_columns(cp_aliased)

{'Actividad Institucional',
 'Adefas',
 'Aprobado',
 'Ciclo',
 'Clave de cartera',
 'Descripción de Fuente de Financiamiento',
 'Descripción de Función',
 'Descripción de Grupo Funcional',
 'Descripción de Objeto del Gasto',
 'Descripción de Programa Presupuestario',
 'Descripción de Ramo',
 'Descripción de Subfunción',
 'Descripción de Tipo de Gasto',
 'Descripción de Unidad Responsable',
 'Descripción de la Actividad Institucional',
 'Descripción de la entidad federativa',
 'Descripción de la modalidad del programa presupuestario',
 'Devengado',
 'Ejercicio',
 'Ejercido',
 'Entidad Federativa',
 'Fuente de Financiamiento',
 'Función',
 'Grupo Funcional',
 'Modalidad del Programa presupuestario',
 'Modificado',
 'Objeto del Gasto',
 'Pagado',
 'Programa Presupuestario',
 'Ramo',
 'Subfunción',
 'Tipo de Gasto',
 'Unidad Responsable'}

In [28]:
cp_columns_overview = build_overview(cp_aliased)
cp_columns_overview

,Column,2010,2011,2012,2013,2014,2015
0,Descripción de Fuente de Financiamiento,True,True,True,True,True,True
1,Descripción de la entidad federativa,False,False,True,True,True,True
2,Subfunción,True,True,True,True,True,True
3,Descripción de Subfunción,True,True,True,True,True,True
4,Programa Presupuestario,True,True,True,True,True,True
5,Entidad Federativa,False,False,True,True,True,True
6,Descripción de Tipo de Gasto,True,True,True,True,True,True
7,Adefas,False,False,False,False,True,True
8,Objeto del Gasto,True,True,True,True,True,True
9,Ciclo,True,True,True,True,True,True


In [29]:
cp_columns_overview.to_csv('cuenta_publica.columns.aliased.csv', index=False, encoding='utf-8')

In [42]:
for year in range(2010, 2016):
    print(year, pef[year]['Importe Presupuesto de Egresos de la Federación (PEF)'].values[310:320])
    

2010 [  4479672.  14107852.   1906255.  11651929.   5830725.   8594754.
   2929711.    185626.  17180266.    868916.]
2011 [  1.11655000e+07   1.55100000e+06   1.95500000e+06   2.20000000e+05
   1.43004490e+07   1.00525941e+08   1.08991620e+07   1.13639540e+07
   4.50074500e+06   1.41332920e+07]
2012 [  4.02913800e+06   6.23900000e+03   2.32723400e+06   1.08001760e+07
   1.20000000e+06   7.41433080e+07   6.53697000e+06   5.01713000e+06
   3.27000000e+05   1.56606000e+05]
2013 [ 19287033.  12956041.    400000.   7897154.  31330606.   5429778.
  10313662.   3198428.   6253310.   1752530.]
2014 ['8,249,300.00' '8,358,497.00' '9,186,700.00' '10,000,000.00'
 '10,486,812.00' '10,905,700.00' '11,074,900.00' '12,576,100.00'
 '14,569,930.00' '15,097,600.00']
2015 ['$2,721,488.00' '$2,835,070.00' '$9,848,067.00' '$375,150.00'
 '$1,032,264.00' '$14,719,440.00' '$8,600.00' '$13,200.00' '$10,590,203.00'
 '$12,968,218.00']


In [49]:
for year in range(2010, 2016):
    for column in cp[year].columns:
        print(year, column, cp[year][column].value_counts()[:10])

2010 Ciclo 2010    129014
Name: Ciclo, dtype: int64
2010 Ramo 9     21003
16    16052
11    12949
12     9150
8      9090
20     8116
10     5989
6      5325
17     4821
14     3815
Name: Ramo, dtype: int64
2010 Descripción de Ramo Comunicaciones y Transportes                                      21003
Medio Ambiente y Recursos Naturales                               16052
Educación Pública                                                 12949
Salud                                                              9150
Agricultura, Ganadería, Desarrollo Rural, Pesca y Alimentación     9090
Desarrollo Social                                                  8116
Economía                                                           5989
Hacienda y Crédito Público                                         5325
Procuraduría General de la República                               4821
Trabajo y Previsión Social                                         3815
Name: Descripción de Ramo, dtype: int64
2010 Uni

KeyError: 10.0

In [51]:
for year in range(2008, 2017):
    for column in pef[year].columns:
        print(year, column, pef[year][column].values[:10])


2008 Ciclo [ 2008.  2008.  2008.  2008.  2008.  2008.  2008.  2008.  2008.  2008.]
2008 Ramo [1 1 1 1 1 1 1 1 1 1]
2008 Descripción de Ramo ['Poder Legislativo' 'Poder Legislativo' 'Poder Legislativo'
 'Poder Legislativo' 'Poder Legislativo' 'Poder Legislativo'
 'Poder Legislativo' 'Poder Legislativo' 'Poder Legislativo'
 'Poder Legislativo']
2008 Unidad Responsable ['100' '100' '100' '100' '100' '100' '100' '100' '100' '100']
2008 Descripción de Unidad Responsable ['H. Cámara de Diputados' 'H. Cámara de Diputados' 'H. Cámara de Diputados'
 'H. Cámara de Diputados' 'H. Cámara de Diputados' 'H. Cámara de Diputados'
 'H. Cámara de Diputados' 'H. Cámara de Diputados' 'H. Cámara de Diputados'
 'H. Cámara de Diputados']
2008 Grupo Funcional [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
2008 Descripción de Grupo Funcional ['Gobierno' 'Gobierno' 'Gobierno' 'Gobierno' 'Gobierno' 'Gobierno'
 'Gobierno' 'Gobierno' 'Gobierno' 'Gobierno']
2008 Función [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
2008 Des